In [1]:
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing import image
import os
import numpy as np
import tensorflow as tf
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt 
from keras.preprocessing import image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
import os
import soundfile as sf
import numpy as np
import librosa
from skimage.transform import resize
import pandas as pd
from tqdm import tqdm


mel_train = []
cqt_train = []
mel_test = []
cqt_test = []
label_train = []
label_test = []

2024-04-29 00:38:20.491468: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 00:38:20.523553: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 00:38:21.054224: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def load_audio_files(audio_folder):
    audio_files = []
    labels = []

    for root, dirs, files in os.walk(audio_folder):
        for file in files:
            if file.endswith(".wav"):
                audio_files.append(os.path.join(root, file))
                labels.append(os.path.basename(root))

    return audio_files, labels
def process_audio(file_path):
    sample_rate = 32000
    n_mfcc = 13
    n_fft = 2048
    hop_length = 512

    # Load audio file
    audio, sample_rate = librosa.load(file_path, sr=sample_rate)

    # Compute Mel spectrogram
    mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=128)
    # Compute MFCC from the Mel spectrogram
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=n_mfcc)

    # Resize spectrograms
    mfcc_resized = resize(mfcc, (224, 224, 3), mode='reflect', anti_aliasing=True)
    mel_resized = resize(mel, (224, 224, 3), mode='reflect', anti_aliasing=True)

    # Clear memory
    del audio

    return mfcc_resized, mel_resized
def process_audio_files(audio_files, audio_folder):
    mfcc_train = []
    mel_train = []
    y_train = []

    total_files = len(audio_files)

    with tqdm(total=total_files, ncols=80) as pbar:
        for file_path in audio_files:
            if file_path.endswith(".wav"):
                # Process audio and extract spectrograms
                mfcc, mel = process_audio(file_path)
                # Append spectrograms and label to the train arrays
                mfcc_train.append(mfcc)
                mel_train.append(mel)
                # Extract label from subfolder name
                label = os.path.basename(os.path.dirname(file_path))
                y_train.append(label)
                pbar.set_postfix({'file': file_path})
                pbar.update()

    # Convert train arrays to NumPy arrays
    mfcc_train = np.array(mfcc_train)
    mel_train = np.array(mel_train)
    y_train = np.array(y_train)

    return mfcc_train, mel_train, y_train
def main(audio_folder):
    audio_files, labels = load_audio_files(audio_folder)
    X_mfcc, X_mel, y = process_audio_files(audio_files, audio_folder)
    return X_mfcc, X_mel, y


In [15]:
# Usage
audio_folder = '../data/raw/'
mfcc_test, mel_test, label_test = main(audio_folder)

100%|█| 514/514 [00:12<00:00, 40.81it/s, file=../data/raw/tired/volume_adjustmen


In [16]:
# Usage
audio_folder = '../data/raw/'
mfcc_train, mel_train, label_train = main(audio_folder)

100%|█| 514/514 [00:13<00:00, 39.52it/s, file=../data/raw/tired/volume_adjustmen


In [52]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder on your label data
label_encoder.fit(label_train)

# Transform the labels to numerical values
y_train_encoded = label_encoder.transform(label_train)
y_test_encoded = label_encoder.transform(label_test)

# Get the unique class names
class_names = label_encoder.classes_

# Create a dictionary to map numerical labels to class names
label_to_class = {i: class_name for i, class_name in enumerate(class_names)}

# Convert the encoded labels to one-hot encoded vectors
label_train_encoded = to_categorical(y_train_encoded)
label_test_encoded = to_categorical(y_test_encoded)


NameError: name 'X_train' is not defined

In [18]:
from tensorflow.keras.utils import to_categorical

mel_train_norm = np.array(mel_train) / 255  
mel_test_norm = np.array(mel_test) / 255      
mfcc_train_norm = np.array(mfcc_train) / 255    
mfcc_test_norm = np.array(mfcc_test) / 255    

In [19]:
# Define input layers
mel_input = Input(shape=(224, 224, 3))  # Modify the shape according to your mel spectrogram dimensions
cqt_input = Input(shape=(224, 224, 3))

# Convolutional layers for Mel spectrograms
mel_conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(mel_input)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Conv2D(64, kernel_size=(3, 3), activation='relu')(mel_conv)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Conv2D(128, kernel_size=(3, 3), activation='relu')(mel_conv)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Flatten()(mel_conv)

# Convolutional layers for CQT spectrograms
cqt_conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(cqt_input)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Conv2D(64, kernel_size=(3, 3), activation='relu')(cqt_conv)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Conv2D(128, kernel_size=(3, 3), activation='relu')(cqt_conv)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Flatten()(cqt_conv)

# Merge the branches
merged = concatenate([mel_conv, cqt_conv])

# Dense layers for classification
dense = Dense(256, activation='relu')(merged)
dense = Dropout(0.5)(dense)  # Add dropout with a rate of 0.5
dense = Dense(128, activation='relu')(dense)
dense = Dropout(0.5)(dense)  # Add dropout with a rate of 0.5
output = Dense(5, activation='softmax')(dense)

# Define the model
model = Model(inputs=[mel_input, cqt_input], outputs=output)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [20]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
earlystop_callback = EarlyStopping(
    monitor='val_accuracy', patience=5, mode='max', verbose=1, restore_best_weights=True
)



In [21]:
hist = model.fit([mel_train_norm, mfcc_train_norm], label_train_encoded,
           batch_size=32, epochs=100,
          validation_data=([mel_test_norm, mfcc_test_norm], label_test_encoded),
          callbacks=[earlystop_callback]
          )

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 17s 939ms/step - accuracy: 0.2133 - loss: 2.0326 - val_accuracy: 0.3132 - val_loss: 1.4761
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 949ms/step - accuracy: 0.3560 - loss: 1.4601 - val_accuracy: 0.4669 - val_loss: 1.2069
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 957ms/step - accuracy: 0.3941 - loss: 1.3130 - val_accuracy: 0.6362 - val_loss: 0.9974
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 965ms/step - accuracy: 0.5092 - loss: 1.1991 - val_accuracy: 0.6809 - val_loss: 0.8705
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 967ms/step - accuracy: 0.5896 - loss: 1.0232 - val_accuracy: 0.7840 - val_loss: 0.6756
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 955ms/step - accuracy: 0.7103 - loss: 0.8867 - val_accuracy: 0.8813 - val_loss: 0.6077
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 958ms/step - accuracy: 0.7097 - loss: 0.7737 - val_accuracy: 0.9222 - val_loss: 0.4195
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 957ms/step - accuracy: 0.8189 - loss: 0.5871 - 

In [16]:
hist

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred = model.predict([mel_test_norm, mfcc_test_norm])

# Convert the predictions from one-hot encoded format to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(label_test_encoded, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_true_labels, y_pred_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')


# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step
Accuracy: 0.9961089494163424
Precision: 0.9961727371308285
Recall: 0.9961089494163424
F1 Score: 0.9961090984685339


In [51]:
# Load the audio file
audio_file_path = '/home/kotaro/Music/babyTest.wav'
#audio_file_path = '../data/raw/discomfort/noise_injection_999bf14b-e417-4b44-b746-9253f81efe38-1430844958178-1.7-m-04-ch_augmented.wav'


# Process the audio to extract features
mfcc, mel = process_audio(audio_file_path)

# Normalize the features
mfcc_norm = np.array([mfcc]) / 255
mel_norm = np.array([mel]) / 255

# Make predictions
prediction = model.predict([mel_norm, mfcc_norm])

# Convert prediction from one-hot encoded to class label
predicted_label_index = np.argmax(prediction)
predicted_class_label = label_to_class[predicted_label_index]

# Display the predicted class label
print("Predicted class label:", predicted_class_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class label: hungry


In [42]:
from collections import Counter

# Count the occurrences of each class label in the original dataset
class_counts = Counter(label_train)

# Print the class counts
for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Count: {count}")


Class: burping, Count: 40
Class: hungry, Count: 139
Class: belly_pain, Count: 80
Class: discomfort, Count: 135
Class: tired, Count: 120
